In [1]:
%load_ext autoreload
%autoreload 2

import os, sys

sys.path.insert(0, "src")

from pandas import DataFrame
from analytics.metrics.metric import MetricCalculator
from analytics.metrics.time import TimeMetrics
from spec.enums import MainTableColumns as Cols
from spec.spec_definition import PS2Versions

In [6]:
from database.config import PS2DataConfig

spec = PS2Versions.v1_0.load()

data_dir = "sample_data/cwo/s19"
config = os.path.join(data_dir, "data_config.yaml")

data_config = PS2DataConfig.from_yaml(config, spec)
data_config.main_table_file


'MainTable.csv'

In [7]:
from database.writer.db_writer_factory import IOFactory
factory = IOFactory.create_factory(data_config)
with factory.create_reader() as reader:
    reader = reader

In [9]:
reader.get_main_table()

,Order,SubjectID,ToolInstances,ServerTimestamp,ServerTimezone,CourseID,CourseSectionID,AssignmentID,ProblemID,Attempt,CodeStateID,IsEventOrderingConsistent,EventType,Score,Compile.Result,CompileMessageType,CompileMessageData,EventID,ParentEventID,SourceLocation
0,0,f40a635828e2bffd0a598a7ed621fc93,Java 8; CodeWorkout,2019-03-20T19:34:25,0,CS 1,1,492.0,32,25,548938774dc948448b0bb51139bbcae3a3966ab8,True,Run.Program,1.000000,NaN,NaN,NaN,32-84779,NaN,NaN
1,1,f40a635828e2bffd0a598a7ed621fc93,Java 8; CodeWorkout,2019-03-20T19:34:25,0,CS 1,1,492.0,32,25,548938774dc948448b0bb51139bbcae3a3966ab8,True,Compile,NaN,Success,NaN,NaN,32-84780,32-84779,NaN
2,2,f18374bf891c9af64c8570448824202d,Java 8; CodeWorkout,2019-03-20T21:02:00,0,CS 1,2,492.0,32,1,20dc599067ac07408b5636973fc6b8a37db9af32,True,Run.Program,1.000000,NaN,NaN,NaN,32-84781,NaN,NaN
3,3,f18374bf891c9af64c8570448824202d,Java 8; CodeWorkout,2019-03-20T21:02:00,0,CS 1,2,492.0,32,1,20dc599067ac07408b5636973fc6b8a37db9af32,True,Compile,NaN,Success,NaN,NaN,32-84782,32-84781,NaN
4,4,4d230b683bf9840553ae57f4acc96e81,Java 8; CodeWorkout,2019-03-20T21:13:09,0,CS 1,1,492.0,32,1,23c57b36002c4d482df3d25f97778f74a2ed581a,True,Run.Program,0.000000,NaN,NaN,NaN,32-84783,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201565,201565,194309a52da9de185b531cfc697cfca8,Java 8; CodeWorkout,2019-03-09T04:57:22,0,CS 1,2,487.0,20,2,f32e2f805ed4bf470905bea0e75da2a08eac1db0,True,Compile,NaN,Success,NaN,NaN,20-14497,20-14496,NaN
201566,201566,194309a52da9de185b531cfc697cfca8,Java 8; CodeWorkout,2019-03-09T04:58:06,0,CS 1,2,487.0,20,3,e005651612dcbc9fabe9717995da83f63188a670,True,Run.Program,0.785714,NaN,NaN,NaN,20-14498,NaN,NaN
201567,201567,194309a52da9de185b531cfc697cfca8,Java 8; CodeWorkout,2019-03-09T04:58:06,0,CS 1,2,487.0,20,3,e005651612dcbc9fabe9717995da83f63188a670,True,Compile,NaN,Success,NaN,NaN,20-14499,20-14498,NaN
201568,201568,194309a52da9de185b531cfc697cfca8,Java 8; CodeWorkout,2019-03-09T04:58:39,0,CS 1,2,487.0,20,4,57e693caa7793b0e0a0800db91738e3a1d5d360c,True,Run.Program,1.000000,NaN,NaN,NaN,20-14500,NaN,NaN


In [ ]:

original_df = DataFrame()

time_metrics = TimeMetrics()

results = time_metrics.calculate(original_df.groupby([Cols.SubjectID, Cols.ProblemID]))